# Doomsayer Tutorial

## Overview

This tutorial notebook contains several demos of how to use Doomsayer.



### Display doomsayer's help prompt

To list all available options for Doomsayer, use the following command:

In [ ]:
!python doomsayer.py -h

### Download example VCFs+reference genome

For the examples provided in this tutorial, you will need to download the following files from the Google Cloud Genomics storage repository

- gzipped VCF files for Chr21 and Chr22 from the 1000 Genomes Phase 3 project
- GRCh37-lite reference genome

In [ ]:
!wget https://storage.googleapis.com/genomics-public-data/ftp-trace.ncbi.nih.gov/1000genomes/ftp/release/20130502/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz
!wget https://storage.googleapis.com/genomics-public-data/ftp-trace.ncbi.nih.gov/1000genomes/ftp/release/20130502/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz
!wget https://storage.googleapis.com/genomics-public-data/references/GRCh37lite/GRCh37-lite.fa.gz
!gunzip GRCh37-lite.fa.gz

## Run Doomsayer on single VCF

The default input for Doomsayer is a single vcf file. The following examples run Doomsayer separately on the 1000 Genomes VCFs for Chromosome 21 and Chromosome 22. To keep the output from both chromosomes, we use the `--projectdir "1kg_chr[N]"` option.

In [ ]:
!python doomsayer.py \
    --mode vcf \
    --input ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz \
    --fastafile GRCh37-lite.fa \
    --projectdir "1kg_chr21" \
    --verbose

In [ ]:
!python doomsayer.py \
    --mode vcf \
    --input ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz \
    --fastafile GRCh37-lite.fa \
    --projectdir "1kg_chr22" \
    --verbose

## Run Doomsayer in a pipeline

In VCF mode, Doomsayer will accept input piped from other programs by specifying `--input -`, allowing the user to perform any number of pre-processing steps to the VCF file.

In the example below, we are pre-filtering the Chr22 VCF with bcftools to include only biallelic SNVs

In [ ]:
!bcftools view -m2 -M2 -v snps ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz | \
python doomsayer.py \
  --mode vcf \
  --input - \
  --fastafile GRCh37-lite.fa \
  --verbose

## Run Doomsayer on multiple VCFs

In VCF mode, Doomsayer will also accept a text file containing the file paths of the VCFs we wish to process (one per line).

This example runs with the `--cpus` option to enable parallel processing of the VCFs.

(this is run without the `--verbose` flag)

In [ ]:
# create input text file
!ls *.vcf.gz > vcfs.txt

# confirm text file has the VCFs we want
!cat vcfs.txt

# run doomsayer with VCF list as input
!python doomsayer.py \
    --mode vcf \
    --input vcfs.txt \
    --fastafile GRCh37-lite.fa \
    --projectdir "1kg_combined" \
    --cpus 2

## Automatically create filtered output

In VCF or text mode, when you specify the `--filterout` option, once the list of outliers has been generated, Doomsayer will create a new VCF with all of the outliers removed. This output is written to `STDOUT` to enable piping to other processing steps; to write to a file, simply add `> output.vcf` at the end of the command.

In [ ]:
!python doomsayer.py \
  --mode vcf \
  --input ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz \
  --fastafile GRCh37-lite.fa \
  --verbose \
  --rank 4 \
  --filterout  > chr22.autofiltered.vcf

## Run Doomsayer on an existing count matrix

In some cases, you may wish to re-run the outlier detection or mutation signature analysis using different options (e.g., with a different rank). To avoid having to re-generate the input matrix, you can use Doomsayer in aggregation mode (`--mode agg`), and supply an existing matrix as the input. Here we are using the count matrix we previously generated from the Chr21 VCF.

In [ ]:
!python doomsayer.py \
  --mode agg \
  --input "1kg_chr21/NMF_M_spectra.txt" \
  --verbose \
  --rank 4

## Aggregation mode with multiple input matrices

You may also encounter situations where you have generated count matrices on partitions of your data and need to recombine them. Doomsayer can aggregate data in two ways:

- combining from runs on data split by genomic regions (but in identical samples)
- combining from runs on data split by different samples

### Aggregating output from different regions

If you need to combine output from multiple previous Doomsayer runs on non-overlapping genomic regions, create a text file named `m_regions.txt` containing the paths to the count matrices (one per line), and use as the input file in aggregation mode (`--mode agg`).

This mode is particularly useful if you are working with very large datasets (>10,000 samples), as you can split the input into small regions and spawn many simultaneous Doomsayer runs using Slurm or other workload managers. 

(though it is often easier to run in multiple VCF mode rather than writing output )

Note that the input matrices **must** have identical dimensions. **Samples must be ordered identically in the rows of each matrix.**

In [ ]:
# create text file containing paths of count matrices from the chr21 and chr22 output
!find . -type d -name "1kg_chr*" -exec find {} -type f -name "*spectra.txt" \; > m_regions.txt

# confirm text file has the count matrices we want
!cat m_regions.txt

# if input file is named "m_regions.txt," doomsayer will add matrices element-wise
!python doomsayer.py \
  --mode agg \
  --input m_regions.txt \
  --verbose \
  --rank 4

### Aggregating output from different subsamples

In some instances, your data may be split into subsamples--Doomsayer can be run on these separately to generate the subsample count matrices, and then run again in aggregation mode to concatenate these matrices. To aggregate in this way, create a text file named `m_samples.txt` containing the paths to the subsample count matrices (one per line), and use as the input file in aggregation mode (`--mode agg`).

We recommend caution when using this mode; data should usually be split into subsamples only after variant calling has been performed over the entire sample.

In [ ]:
# create text file containing paths of count matrices from the chr21 and chr22 output
!find . -type d -name "1kg_chr*" -exec find {} -type f -name "*spectra.txt" \; > m_samples.txt

# confirm text file has the count matrices we want
!cat m_samples.txt

# if input file is named "m_samples.txt," doomsayer will concatenate matrices rowwise
# i.e., if matrix 1 is N1xK and matrix 2 is N2xK, the new matrix will be (N1+N2)xK
!python doomsayer.py \
  --mode agg \
  --input m_samples.txt \
  --verbose \
  --rank 4

## Generate diagnostic report

In this example, we run Doomsayer on an existing count matrix, previously generated from the entire 1000 Genomes data, and include the `--report` option to generate a diagnostic report in the output directory.

In [ ]:
!python doomsayer.py \
  --mode agg \
  --input tutorial_data/NMF_M_spectra.txt \
  --verbose \
  --rank 4 \
  --report